In [ ]:
# primero establecer el Runtime de Python 3
from google.colab import drive
drive.mount('/content/.drive')

Drive already mounted at /content/.drive; to attempt to forcibly remount, call drive.mount("/content/.drive", force_remount=True).


In [ ]:
%%shell

mkdir -p "/content/.drive/My Drive/dmeyf"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/dmeyf" /content/buckets/b1


mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets



archivo_origen="https://storage.googleapis.com/open-courses/dmeyf2025-e4a2/competencia_01_crudo.csv"
archivo_destino="/content/datasets/competencia_01_crudo.csv"
archivo_destino_bucket="/content/buckets/b1/datasets/competencia_01_crudo.csv"

if ! test -f $archivo_destino_bucket; then
  wget  $archivo_origen  -O $archivo_destino_bucket
fi


if ! test -f $archivo_destino; then
  cp  $archivo_destino_bucket  $archivo_destino
fi


## 1 Dataset

### 1.1 Feature Engineering

### 1.1.1 Generacion de la clase_ternaria

Esta parte se debe correr con el runtime en lenguaje **R** Ir al menu, Runtime -> Change Runtime Tipe -> Runtime type -> R

In [ ]:
require( "data.table" )

# leo el dataset
dataset <- fread("/content/datasets/competencia_01_crudo.csv" )

# calculo el periodo0 consecutivo
dsimple <- dataset[, list(
    "pos" = .I,
    numero_de_cliente,
    periodo0 = as.integer(foto_mes/100)*12 +  foto_mes%%100 ) ]


# ordeno
setorder( dsimple, numero_de_cliente, periodo0 )

# calculo topes
periodo_ultimo <- dsimple[, max(periodo0) ]
periodo_anteultimo <- periodo_ultimo - 1


# calculo los leads de orden 1 y 2
dsimple[, c("periodo1", "periodo2") :=
    shift(periodo0, n=1:2, fill=NA, type="lead"),  numero_de_cliente ]

# assign most common class values = "CONTINUA"
dsimple[ periodo0 < periodo_anteultimo, clase_ternaria := "CONTINUA" ]

# calculo BAJA+1
dsimple[ periodo0 < periodo_ultimo &
    ( is.na(periodo1) | periodo0 + 1 < periodo1 ),
    clase_ternaria := "BAJA+1" ]

# calculo BAJA+2
dsimple[ periodo0 < periodo_anteultimo & (periodo0+1 == periodo1 )
    & ( is.na(periodo2) | periodo0 + 2 < periodo2 ),
    clase_ternaria := "BAJA+2" ]


# pego el resultado en el dataset original y grabo
setorder( dsimple, pos )
dataset[, clase_ternaria := dsimple$clase_ternaria ]

fwrite( dataset,
    file =  "/content/datasets/competencia_01.csv.gz",
    sep = ","
)

Loading required package: data.table



In [ ]:
setorder( dataset, foto_mes, clase_ternaria, numero_de_cliente)
dataset[, .N, list(foto_mes, clase_ternaria)]

foto_mes,clase_ternaria,N
<int>,<chr>,<int>
202101,BAJA+1,622
202101,BAJA+2,825
202101,CONTINUA,160080
202102,BAJA+1,831
202102,BAJA+2,1032
202102,CONTINUA,160292
202103,BAJA+1,1039
202103,BAJA+2,951
202103,CONTINUA,161119


### 1.1.3 Creación de lags y delta lags

In [ ]:
max_lag <- 3

# selecciono automáticamente todas las columnas numéricas, excepto id y fecha
campos_laguear <- setdiff(
  names(dataset)[sapply(dataset, is.numeric)],
  c("numero_de_cliente", "foto_mes")
)

cat("Máximo lag posible:", max_lag, "\n")
cat("Campos a laggear:", paste(campos_laguear, collapse=", "), "\n")

# genero los lags (ya lo tenías)
dataset[, paste0(rep(campos_laguear, each = max_lag), "_lag", 1:max_lag) :=
          shift(.SD, n = 1:max_lag, type = "lag"),
        by = numero_de_cliente,
        .SDcols = campos_laguear]

# genero los delta lags como var - lagX
for (campo in campos_laguear) {
  for (k in 1:max_lag) {
    dataset[, (paste0(campo, "_delta", k)) :=
              get(campo) - get(paste0(campo, "_lag", k))]
  }
}

Máximo lag posible: 3 
Campos a laggear: active_quarter, cliente_vip, internet, cliente_edad, cliente_antiguedad, mrentabilidad, mrentabilidad_annual, mcomisiones, mactivos_margen, mpasivos_margen, cproductos, tcuentas, ccuenta_corriente, mcuenta_corriente_adicional, mcuenta_corriente, ccaja_ahorro, mcaja_ahorro, mcaja_ahorro_adicional, mcaja_ahorro_dolares, cdescubierto_preacordado, mcuentas_saldo, ctarjeta_debito, ctarjeta_debito_transacciones, mautoservicio, ctarjeta_visa, ctarjeta_visa_transacciones, mtarjeta_visa_consumo, ctarjeta_master, ctarjeta_master_transacciones, mtarjeta_master_consumo, cprestamos_personales, mprestamos_personales, cprestamos_prendarios, mprestamos_prendarios, cprestamos_hipotecarios, mprestamos_hipotecarios, cplazo_fijo, mplazo_fijo_dolares, mplazo_fijo_pesos, cinversion1, minversion1_pesos, minversion1_dolares, cinversion2, minversion2, cseguro_vida, cseguro_auto, cseguro_vivienda, cseguro_accidentes_personales, ccaja_seguridad, cpayroll_trx, mpayroll, mp

In [ ]:
fwrite( dataset,
    file =  "/content/datasets/competencia_01.csv.gz",
    sep = ","
)

### 2.2 Optimizacion Hiperparámetros

Esta parte se debe correr con el runtime en lenguaje R Ir al menu, Runtime -> Change Runtime Type -> Runtime type -> R

### 2.2.1 Inicio

limpio el ambiente de R

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Oct 10 04:11:37 AM 2025"

In [ ]:
# limpio la memoria
rm(list=ls(all.names=TRUE)) # remove all objects
gc(full=TRUE, verbose=FALSE) # garbage collection

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,730074,39.0,1454466,77.7,1454466,77.7
Vcells,1388503,10.6,898494184,6855.0,779340483,5945.9


### 2.2.2 Carga de Librerias

Esta parte lleva varios minutos la primera vez en Google Colab

In [ ]:
# cargo las librerias que necesito
require("data.table")
require("parallel")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

if( !require("primes") ) install.packages("primes")
require("primes")

if( !require("utils") ) install.packages("utils")
require("utils")

if( !require("rlist") ) install.packages("rlist")
require("rlist")

if( !require("yaml")) install.packages("yaml")
require("yaml")

if( !require("lightgbm") ) install.packages("lightgbm")
require("lightgbm")

if( !require("DiceKriging") ) install.packages("DiceKriging")
require("DiceKriging")

if( !require("mlrMBO") ) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: parallel

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.


At

### 2.2.3 Definicion de Parametros

aqui debe cargar SU semilla primigenia
<br>recuerde cambiar el numero de experimento en cada corrida nueva

#### Nombre experimento y semilla

In [ ]:
PARAM <- list()
PARAM$experimento <- 'C01-38d'
PARAM$semilla_primigenia <- 170029


#### Seteo meses

In [ ]:
# training y future
PARAM$train <- c(202101, 202102)  # Para BO
PARAM$train_final <- c(202101, 202102, 202103, 202104)  # Para modelo final
PARAM$future <- c(202106)  # CAMBIO PRINCIPAL
PARAM$test <- c(202104)  # Nuevo: para validar hiperparámetros
PARAM$semilla_kaggle <- 314159
PARAM$cortes <- seq(6000, 19000, by= 500)

#### Undersampling

In [ ]:
# un undersampling de 0.1  toma solo el 10% de los CONTINUA
# undersampling de 1.0  implica tomar TODOS los datos

PARAM$trainingstrategy$undersampling <- 0.1

In [ ]:
# Parametros LightGBM

PARAM$hyperparametertuning$xval_folds <- 5

# parametros fijos del LightGBM que se pisaran con la parte variable de la BO
PARAM$lgbm$param_fijos <-  list(
  boosting= "gbdt", # puede ir  dart  , ni pruebe random_forest
  objective= "binary",
  metric= "auc",
  first_metric_only= FALSE,
  boost_from_average= TRUE,
  feature_pre_filter= FALSE,
  force_row_wise= TRUE, # para reducir warnings
  verbosity= -100,

  seed= PARAM$semilla_primigenia,

  max_depth= -1L, # -1 significa no limitar,  por ahora lo dejo fijo
  min_gain_to_split= 0, # min_gain_to_split >= 0
  min_sum_hessian_in_leaf= 0.001, #  min_sum_hessian_in_leaf >= 0.0
  lambda_l1= 0.0, # lambda_l1 >= 0.0
  lambda_l2= 0.0, # lambda_l2 >= 0.0
  max_bin= 31L, # lo debo dejar fijo, no participa de la BO

  bagging_fraction= 1.0, # 0.0 < bagging_fraction <= 1.0
  pos_bagging_fraction= 1.0, # 0.0 < pos_bagging_fraction <= 1.0
  neg_bagging_fraction= 1.0, # 0.0 < neg_bagging_fraction <= 1.0
  is_unbalance= FALSE, #
  scale_pos_weight= 1.0, # scale_pos_weight > 0.0

  drop_rate= 0.1, # 0.0 < neg_bagging_fraction <= 1.0
  max_drop= 50, # <=0 means no limit
  skip_drop= 0.5, # 0.0 <= skip_drop <= 1.0

  extra_trees= FALSE,

  num_iterations= 1200,
  learning_rate= 0.02,
  feature_fraction= 0.5,
  num_leaves= 750,
  min_data_in_leaf= 5000
)


Aqui se definen los hiperparámetros de LightGBM que participan de la Bayesian Optimization
<br> si es un numero entero debe ir  makeIntegerParam
<br> si es un numero real (con decimales) debe ir  makeNumericParam
<br> es muy importante leer cuales son un lower y upper  permitidos y ademas razonables

In [ ]:
# Aqui se cargan los bordes de los hiperparametros de la BO
PARAM$hypeparametertuning$hs <- makeParamSet(
  # Iteraciones / tasa de aprendizaje
  makeIntegerParam("num_iterations", lower = 8L, upper = 2048L),
  makeNumericParam("learning_rate", lower = 0.005, upper = 0.3),

  # Complejidad del árbol
  makeIntegerParam("num_leaves", lower = 8L, upper = 2048L),
  makeIntegerParam("max_depth", lower = -1L, upper = 30L),
  makeIntegerParam("min_data_in_leaf", lower = 1L, upper = 6000L),
  makeNumericParam("min_split_gain", lower = 0.0, upper = 0.3),

  # Submuestreo de features y filas
  makeNumericParam("feature_fraction", lower = 0.1, upper = 1.0),
  makeNumericParam("bagging_fraction", lower = 0.5, upper = 1.0),
  makeIntegerParam("bagging_freq", lower = 1L, upper = 10L),

  # Regularización L1/L2
  makeNumericParam("lambda_l1", lower = 0.0, upper = 5.0),
  makeNumericParam("lambda_l2", lower = 0.0, upper = 5.0)


)


A mayor cantidad de hiperparámetros, se debe aumentar las iteraciones de la Bayesian Optimization
<br> 30 es un valor muy tacaño, pero corre rápido
<br> deberia partir de 50, alcanzando los 100 si se dispone de tiempo

In [ ]:
PARAM$hyperparametertuning$iteraciones <- 30 # iteraciones bayesianas

In [ ]:
# particionar agrega una columna llamada fold a un dataset
#   que consiste en una particion estratificada segun agrupa
# particionar( data=dataset, division=c(70,30),
#  agrupa=clase_ternaria, seed=semilla)   crea una particion 70, 30

particionar <- function(data, division, agrupa= "", campo= "fold", start= 1, seed= NA) {
  if (!is.na(seed)) set.seed(seed, "L'Ecuyer-CMRG")

  bloque <- unlist(mapply(
    function(x, y) {rep(y, x)},division, seq(from= start, length.out= length(division))))

  data[, (campo) := sample(rep(bloque,ceiling(.N / length(bloque))))[1:.N],by= agrupa]
}

In [ ]:
# iniciliazo el dataset de realidad, para medir ganancia
realidad_inicializar <- function( pfuture, pparam) {

  # datos para verificar la ganancia
  drealidad <- pfuture[, list(numero_de_cliente, foto_mes, clase_ternaria)]

  particionar(drealidad,
    division= c(3, 7),
    agrupa= "clase_ternaria",
    seed= PARAM$semilla_kaggle
  )

  return( drealidad )
}

In [ ]:
# evaluo ganancia en los datos de la realidad

realidad_evaluar <- function( prealidad, pprediccion) {

  prealidad[ pprediccion,
    on= c("numero_de_cliente", "foto_mes"),
    predicted:= i.Predicted
  ]

  tbl <- prealidad[, list("qty"=.N), list(fold, predicted, clase_ternaria)]

  res <- list()
  res$public  <- tbl[fold==1 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.3
  res$private <- tbl[fold==2 & predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]/0.7
  res$total <- tbl[predicted==1L, sum(qty*ifelse(clase_ternaria=="BAJA+2", 780000, -20000))]

  prealidad[, predicted:=NULL]
  return( res )
}

### 2.2.4  Preprocesamiento

In [ ]:
# carpeta de trabajo

setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("HT", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento_folder ))

In [ ]:
# lectura del dataset
dataset <- fread("/content/datasets/competencia_01.csv.gz", stringsAsFactors= TRUE)

In [ ]:
dataset_train <- dataset[foto_mes %in% PARAM$train]

In [ ]:
# paso la clase a binaria que tome valores {0,1}  enteros
#  BAJA+1 y BAJA+2  son  1,   CONTINUA es 0
#  a partir de ahora ya NO puedo cortar  por prob(BAJA+2) > 1/40

dataset_train[,
  clase01 := ifelse(clase_ternaria %in% c("BAJA+2","BAJA+1"), 1L, 0L)
]

In [ ]:
# defino los datos que forma parte del training
# aqui se hace el undersampling de los CONTINUA
# notar que para esto utilizo la SEGUNDA semilla

set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset_train[, azar := runif(nrow(dataset_train))]
dataset_train[, training := 0L]

dataset_train[
  foto_mes %in%  PARAM$train &
    (azar <= PARAM$trainingstrategy$undersampling | clase_ternaria %in% c("BAJA+1", "BAJA+2")),
  training := 1L
]

In [ ]:
# los campos que se van a utilizar

campos_buenos <- setdiff(
  colnames(dataset_train),
  c("clase_ternaria", "clase01", "azar", "training")
)

In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain <- lgb.Dataset(
  data= data.matrix(dataset_train[training == 1L, campos_buenos, with= FALSE]),
  label= dataset_train[training == 1L, clase01],
  free_raw_data= FALSE
)

nrow(dtrain)
ncol(dtrain)

### 2.2.5 Configuracion Bayesian Optimization

In [ ]:
# En el argumento x llegan los parmaetros de la bayesiana
#  devuelve la AUC en cross validation del modelo entrenado

EstimarGanancia_AUC_lightgbm <- function(x) {

  # x pisa (o agrega) a param_fijos
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # entreno LightGBM
  modelocv <- lgb.cv(
    data= dtrain,
    nfold= PARAM$hyperparametertuning$xval_folds,
    stratified= TRUE,
    param= param_completo
  )

  # obtengo la ganancia
  AUC <- modelocv$best_score

  # hago espacio en la memoria
  rm(modelocv)
  gc(full= TRUE, verbose= FALSE)

  message(format(Sys.time(), "%a %b %d %X %Y"), " AUC ", AUC)

  return(AUC)
}

In [ ]:
# Aqui comienza la configuracion de la Bayesian Optimization

# en este archivo quedan la evolucion binaria de la BO
kbayesiana <- "bayesiana.RDATA"

funcion_optimizar <- EstimarGanancia_AUC_lightgbm # la funcion que voy a maximizar

configureMlr(show.learner.output= FALSE)

# configuro la busqueda bayesiana,  los hiperparametros que se van a optimizar
# por favor, no desesperarse por lo complejo

obj.fun <- makeSingleObjectiveFunction(
  fn= funcion_optimizar, # la funcion que voy a maximizar
  minimize= FALSE, # estoy Maximizando la ganancia
  noisy= TRUE,
  par.set= PARAM$hypeparametertuning$hs, # definido al comienzo del programa
  has.simple.signature= FALSE # paso los parametros en una lista
)

# cada 600 segundos guardo el resultado intermedio
ctrl <- makeMBOControl(
  save.on.disk.at.time= 600, # se graba cada 600 segundos
  save.file.path= kbayesiana
) # se graba cada 600 segundos

# indico la cantidad de iteraciones que va a tener la Bayesian Optimization
ctrl <- setMBOControlTermination(
  ctrl,
  iters= PARAM$hyperparametertuning$iteraciones
) # cantidad de iteraciones

# defino el método estandar para la creacion de los puntos iniciales,
# los "No Inteligentes"
ctrl <- setMBOControlInfill(ctrl, crit= makeMBOInfillCritEI())

# establezco la funcion que busca el maximo
surr.km <- makeLearner(
  "regr.km",
  predict.type= "se",
  covtype= "matern3_2",
  control= list(trace= TRUE)
)


### 2.2.6 Corrida Bayesian Optimization

In [ ]:
# inicio la optimizacion bayesiana, retomando si ya existe
# es la celda mas lenta de todo el notebook

if (!file.exists(kbayesiana)) {
  bayesiana_salida <- mbo(obj.fun, learner= surr.km, control= ctrl)
} else {
  bayesiana_salida <- mboContinue(kbayesiana) # retomo en caso que ya exista
}

Computing y column(s) for design. Not provided.

Fri Oct 10 04:24:26 AM 2025 AUC 0.924920028128152

Fri Oct 10 04:25:06 AM 2025 AUC 0.923123240004608

Fri Oct 10 04:26:14 AM 2025 AUC 0.927797055810691

Fri Oct 10 04:27:43 AM 2025 AUC 0.920812635908348

Fri Oct 10 04:27:51 AM 2025 AUC 0.911291413066959

Fri Oct 10 04:29:01 AM 2025 AUC 0.925016108405555

Fri Oct 10 04:29:38 AM 2025 AUC 0.926178181719056

Fri Oct 10 04:31:03 AM 2025 AUC 0.929806641554219

Fri Oct 10 04:32:00 AM 2025 AUC 0.922156819588702

Fri Oct 10 04:32:57 AM 2025 AUC 0.912840688167139

Fri Oct 10 04:34:37 AM 2025 AUC 0.918826337960942

Fri Oct 10 04:36:56 AM 2025 AUC 0.922924198528519

Fri Oct 10 04:37:55 AM 2025 AUC 0.917681673329927

Fri Oct 10 04:39:28 AM 2025 AUC 0.926958527514656

Fri Oct 10 04:40:39 AM 2025 AUC 0.917547641720882

Fri Oct 10 04:41:07 AM 2025 AUC 0.925804519353563

Fri Oct 10 04:42:26 AM 2025 AUC 0.927492570819676

Fri Oct 10 04:42:40 AM 2025 AUC 0.921541328364175

Fri Oct 10 04:44:46 AM 2025 AUC 0

In [ ]:

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
colnames( tb_bayesiana)

[1] "num_iterations"   "learning_rate"    "num_leaves"       "max_depth"       
 [5] "min_data_in_leaf" "min_split_gain"   "feature_fraction" "bagging_fraction"
 [9] "bagging_freq"     "lambda_l1"        "lambda_l2"        "y"               
[13] "dob"              "eol"              "error.message"    "exec.time"       
[17] "ei"               "error.model"      "train.time"       "prop.type"       
[21] "propose.time"     "se"               "mean"

In [ ]:
# almaceno los resultados de la Bayesian Optimization
# y capturo los mejores hiperparametros encontrados

tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)

tb_bayesiana[, iter := .I]

# ordeno en forma descendente por AUC = y
setorder(tb_bayesiana, -y)

# grabo para eventualmente poder utilizarlos en OTRA corrida
fwrite( tb_bayesiana,
  file= "BO_log.txt",
  sep= "\t"
)

# los mejores hiperparámetros son los que quedaron en el registro 1 de la tabla
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1, # el primero es el de mejor AUC
  setdiff(colnames(tb_bayesiana),
    c("y","dob","eol","error.message","exec.time","ei","error.model",
      "train.time","prop.type","propose.time","se","mean","iter")),
  with= FALSE
]


PARAM$out$lgbm$y <- tb_bayesiana[1, y]


In [ ]:
write_yaml( PARAM, file="PARAM.yml")

In [ ]:
print(PARAM$out$lgbm$mejores_hiperparametros)
print(PARAM$out$lgbm$y)

   num_iterations learning_rate num_leaves max_depth min_data_in_leaf
            <int>         <num>      <int>     <int>            <int>
1:           1148    0.07171618        652        20                6
   min_split_gain feature_fraction bagging_fraction bagging_freq lambda_l1
            <num>            <num>            <num>        <int>     <num>
1:   0.0006355397        0.9505841        0.9735495            5 0.2978993
   lambda_l2
       <num>
1:  1.631213
[1] 0.9374084


### 2.2.7 Testing

In [ ]:
# Definir las 10 semillas a utilizar
semillas <- c(102191, 202111, 303131, 404243, 505357,
              606469, 707579, 808681, 909793, 1010101)

predicciones_lista <- list()

for(semilla in semillas) {

  cat("\nEntrenando con semilla:", semilla, "\n")

  # Modificar los parámetros para incluir la semilla actual
  param_test <- modifyList(PARAM$lgbm$param_fijos,
                           PARAM$out$lgbm$mejores_hiperparametros)
  param_test$seed <- semilla

  # Entrenar el modelo con la semilla actual
  modelo_test <- lgb.train(
    data = dtrain,
    param = param_test
  )

  # Preparar el dataset de predicción
  dataset_test <- dataset[foto_mes == 202104]

  # Predecir sobre el dataset de test
  prediccion_actual <- predict(
    modelo_test,
    data.matrix(dataset_test[, campos_buenos, with = FALSE])
  )

  # Guardar la predicción en la lista
  predicciones_lista[[as.character(semilla)]] <- prediccion_actual
}


predicciones_promedio <- rowMeans(as.data.table(predicciones_lista))


tb_test <- dataset_test[, .(numero_de_cliente, foto_mes)]
tb_test[, prob := predicciones_promedio] # <-- Usamos el promedio

# Inicializar realidad para test
drealidad_test <- realidad_inicializar(dataset_test, PARAM)

# Calcular ganancias usando realidad_evaluar
resultados_cortes <- data.table()
setorder(tb_test, -prob)

for(envios in PARAM$cortes) {

  tb_test[, Predicted := 0L]
  tb_test[1:envios, Predicted := 1L]

  res <- realidad_evaluar(drealidad_test, tb_test)

  resultados_cortes <- rbind(resultados_cortes,
                             data.table(
                               envios = envios,
                               ganancia_total = res$total,
                               ganancia_public = res$public,
                               ganancia_private = res$private,
                               ganancia_por_envio = res$total / envios
                             ))

  options(scipen = 999)
  cat("Envios=", envios, "\t",
      " TOTAL=", res$total,
      "  Public=", res$public,
      " Private=", res$private,
      "\n",
      sep = "")
}

cat("\nProceso finalizado. Las ganancias mostradas corresponden al promedio de 10 semillas.\n")




Entrenando con semilla: 102191 

Entrenando con semilla: 202111 

Entrenando con semilla: 303131 

Entrenando con semilla: 404243 

Entrenando con semilla: 505357 

Entrenando con semilla: 606469 

Entrenando con semilla: 707579 

Entrenando con semilla: 808681 

Entrenando con semilla: 909793 

Entrenando con semilla: 1010101 
Envios=6000	 TOTAL=340800000  Public=296533333 Private=359771429
Envios=6500	 TOTAL=348400000  Public=302666667 Private=368000000
Envios=7000	 TOTAL=353600000  Public=303333333 Private=375142857
Envios=7500	 TOTAL=362000000  Public=305600000 Private=386171429
Envios=8000	 TOTAL=362400000  Public=301733333 Private=388400000
Envios=8500	 TOTAL=370800000  Public=305333333 Private=398857143
Envios=9000	 TOTAL=372000000  Public=303466667 Private=401371429
Envios=9500	 TOTAL=371600000  Public=299600000 Private=402457143
Envios=10000	 TOTAL=370400000  Public=296800000 Private=401942857
Envios=10500	 TOTAL=366800000  Public=300800000 Private=395085714
Envios=11000	 TOT

In [ ]:
# Encontrar el óptimo
setorder(resultados_cortes, -ganancia_total)
optimo <- resultados_cortes[1]

cat("\n================================================\n")
cat("MEJOR PUNTO DE CORTE EN TEST (202104):\n")
cat("Envíos óptimos:", optimo$envios, "\n")
cat("Ganancia TOTAL:", optimo$ganancia_total, "\n")
cat("Ganancia PUBLIC:", optimo$ganancia_public, "\n")
cat("Ganancia PRIVATE:", optimo$ganancia_private, "\n")
cat("Ganancia por envío:", round(optimo$ganancia_por_envio, 2), "\n")
cat("================================================\n\n")


MEJOR PUNTO DE CORTE EN TEST (202104):
Envíos óptimos: 9000 
Ganancia TOTAL: 372000000 
Ganancia PUBLIC: 303466667 
Ganancia PRIVATE: 401371429 
Ganancia por envío: 41333.33 



## 2.3  Produccion

### Final Training
Construyo el modelo final, que es uno solo, no hace ningun tipo de particion < training, validation, testing>]

In [ ]:
setwd("/content/buckets/b1/exp")
experimento <- paste0("exp", PARAM$experimento)
dir.create(experimento, showWarnings= FALSE)
setwd( paste0("/content/buckets/b1/exp/", experimento ))

#### Final Training Dataset

Aqui esta la gran decision de en qué meses hago el Final Training
<br> debo utilizar los mejores hiperparámetros que encontré en la  optimización bayesiana

In [ ]:
# clase01
dataset[, clase01 := ifelse(clase_ternaria %in% c("BAJA+1", "BAJA+2"), 1L, 0L)]

In [ ]:
dataset_train <- dataset[foto_mes %in% PARAM$train_final]
dataset_train[,.N,clase_ternaria]

clase_ternaria,N
<fct>,<int>
BAJA+1,3447
BAJA+2,3938
CONTINUA,642824


In [ ]:
# dejo los datos en el formato que necesita LightGBM

dtrain_final <- lgb.Dataset(
  data= data.matrix(dataset_train[, campos_buenos, with= FALSE]),
  label= dataset_train[, clase01]
)

#### Final Training Hyperparameters

In [ ]:
param_final <- modifyList(PARAM$lgbm$param_fijos,
  PARAM$out$lgbm$mejores_hiperparametros)

param_final

$boosting
[1] "gbdt"

$objective
[1] "binary"

$metric
[1] "auc"

$first_metric_only
[1] FALSE

$boost_from_average
[1] TRUE

$feature_pre_filter
[1] FALSE

$force_row_wise
[1] TRUE

$verbosity
[1] -100

$seed
[1] 170029

$max_depth
[1] 20

$min_gain_to_split
[1] 0

$min_sum_hessian_in_leaf
[1] 0.001

$lambda_l1
[1] 0.2978993

$lambda_l2
[1] 1.631213

$max_bin
[1] 31

$bagging_fraction
[1] 0.9735495

$pos_bagging_fraction
[1] 1

$neg_bagging_fraction
[1] 1

$is_unbalance
[1] FALSE

$scale_pos_weight
[1] 1

$drop_rate
[1] 0.1

$max_drop
[1] 50

$skip_drop
[1] 0.5

$extra_trees
[1] FALSE

$num_iterations
[1] 1148

$learning_rate
[1] 0.07171618

$feature_fraction
[1] 0.9505841

$num_leaves
[1] 652

$min_data_in_leaf
[1] 6

$min_split_gain
[1] 0.0006355397

$bagging_freq
[1] 5

#### Training
Genero el modelo final, siempre sobre TODOS los datos de  final_train, sin hacer ningun tipo de undersampling de la clase mayoritaria y mucho menos cross validation.

In [ ]:
# este punto es muy SUTIL  y será revisado en la Clase 05

param_normalizado <- copy(param_final)
param_normalizado$min_data_in_leaf <-  round(param_final$min_data_in_leaf / PARAM$trainingstrategy$undersampling)

In [ ]:
# entreno los lightgbm
# Definir las 10 semillas a utilizar

semillas <- c(102191, 202111, 303131, 404243, 505357,
              606469, 707579, 808681, 909793, 1010101)

# Lista para guardar modelos
modelos_finales <- list()

# Entrenar un modelo por cada semilla
for(i in seq_along(semillas)) {

  semilla <- semillas[i]
  cat(sprintf("[%d/%d] Entrenando modelo con semilla: %d\n",
              i, length(semillas), semilla))

  # Modificar parámetros con la semilla actual
  param_actual <- param_normalizado
  param_actual$seed <- semilla

  # Entrenar el modelo
  modelo <- lgb.train(
    data = dtrain_final,
    param = param_actual
  )

  # Guardar en la lista
  modelos_finales[[as.character(semilla)]] <- modelo
}

cat("\n✓ Entrenamiento completado: 10 modelos entrenados\n")

[1/10] Entrenando modelo con semilla: 102191
[2/10] Entrenando modelo con semilla: 202111
[3/10] Entrenando modelo con semilla: 303131
[4/10] Entrenando modelo con semilla: 404243
[5/10] Entrenando modelo con semilla: 505357
[6/10] Entrenando modelo con semilla: 606469
[7/10] Entrenando modelo con semilla: 707579
[8/10] Entrenando modelo con semilla: 808681
[9/10] Entrenando modelo con semilla: 909793
[10/10] Entrenando modelo con semilla: 1010101

✓ Entrenamiento completado: 10 modelos entrenados


In [ ]:
# ahora imprimo la importancia de variables
tb_importancia <- as.data.table(lgb.importance(modelos_finales[[1]]))
archivo_importancia <- "impo.txt"

fwrite(tb_importancia,
  file= archivo_importancia,
  sep= "\t"
)

In [ ]:
# grabo a disco el modelo en un formato para seres humanos ... ponele ...
lgb.save(modelos_finales[[1]], "modelo.txt" )

### Scoring

Aplico el modelo final a los datos del futuro

In [ ]:
# aplico el modelo a los datos sin clase
dfuture <- dataset[foto_mes %in% PARAM$future]

# Matriz de datos para predicción
datos_future <- data.matrix(dfuture[, campos_buenos, with = FALSE])

# Lista para guardar predicciones
predicciones_lista <- list()

# Predecir con cada modelo
for(i in seq_along(semillas)) {
  semilla <- semillas[i]

  prediccion <- predict(
    modelos_finales[[as.character(semilla)]],
    datos_future
  )

  predicciones_lista[[as.character(semilla)]] <- prediccion
}

# PROMEDIO DE LAS 10 PREDICCIONES
prediccion <- rowMeans(as.data.table(predicciones_lista))

In [ ]:
# inicilizo el dataset  drealidad
drealidad <- realidad_inicializar( dfuture, PARAM)

#### Tabla Prediccion

In [ ]:
# tabla de prediccion

tb_prediccion <- dfuture[, list(numero_de_cliente, foto_mes)]
tb_prediccion[, prob := prediccion ]

# grabo las probabilidad del modelo
fwrite(tb_prediccion,
  file= "prediccion.txt",
  sep= "\t"
)

Kaggle Competition Submit

In [ ]:
PARAM$cortes

[1]  6000  6500  7000  7500  8000  8500  9000  9500 10000 10500 11000 11500
[13] 12000 12500 13000 13500 14000 14500 15000 15500 16000 16500 17000 17500
[25] 18000 18500 19000

In [ ]:
# genero archivos con los  "envios" mejores
# suba TODOS los archivos a Kaggle

# ordeno por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle")

for (envios in PARAM$cortes) {

  tb_prediccion[, Predicted := 0L] # seteo inicial a 0
  tb_prediccion[1:envios, Predicted := 1L] # marco los primeros

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  # grabo el archivo
  fwrite(tb_prediccion[, list(numero_de_cliente, Predicted)],
    file= archivo_kaggle,
    sep= ","
  )

  res <- realidad_evaluar( drealidad, tb_prediccion)

  options(scipen = 999)
  cat( "Envios=", envios, "\t",
    " TOTAL=", res$total,
    "  Public=", res$public,
    " Private=", res$private,
    "\n",
    sep= ""
  )

}

Envios=6000	 TOTAL=-120000000  Public=-123266667 Private=-118600000
Envios=6500	 TOTAL=-130000000  Public=-133133333 Private=-128657143
Envios=7000	 TOTAL=-140000000  Public=-144000000 Private=-138285714
Envios=7500	 TOTAL=-150000000  Public=-153666667 Private=-148428571
Envios=8000	 TOTAL=-160000000  Public=-163866667 Private=-158342857
Envios=8500	 TOTAL=-170000000  Public=-174533333 Private=-168057143
Envios=9000	 TOTAL=-180000000  Public=-184733333 Private=-177971429
Envios=9500	 TOTAL=-190000000  Public=-193933333 Private=-188314286
Envios=10000	 TOTAL=-200000000  Public=-204133333 Private=-198228571
Envios=10500	 TOTAL=-210000000  Public=-213200000 Private=-208628571
Envios=11000	 TOTAL=-220000000  Public=-222133333 Private=-219085714
Envios=11500	 TOTAL=-230000000  Public=-232000000 Private=-229142857
Envios=12000	 TOTAL=-240000000  Public=-241733333 Private=-239257143
Envios=12500	 TOTAL=-250000000  Public=-252733333 Private=-248828571
Envios=13000	 TOTAL=-260000000  Public=-26

In [ ]:
write_yaml( PARAM, file="PARAM.yml")

In [ ]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Fri Oct 10 09:44:24 AM 2025"

Finalmente usted deberá cargar el resultado de su corrida en la Google Sheet Colaborativa,  hoja **TareaHogar04**
<br> Siéntase libre de agregar las columnas que hagan falta a la planilla